# Model Tuning: Grid Search + Pipeline

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats as stats

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Objectives

- Explain what hyperparameters are
- Describe the purpose of grid searching
- Implement grid searching for the purposes of model optimization.

# Model Tuning

![](https://imgs.xkcd.com/comics/machine_learning.png)

## Hyperparameters

Many of the models we have looked at are really *families* of models in the sense that they make use of **hyperparameters**.

Thus for example the $k$-nearest-neighbors algorithm allows us to make:

- a 1-nearest-neighbor model
- a 2-nearest-neighbors model
- a 3-nearest-neighbors model
- etc.

Or, for another example, the decision tree algorithm allows us to make:

- a classifier that branches according to information gain
- a classifier that branches according to Gini impurity
- a regressor that branches according to mean squared error
- etc.

Depending on the sort of problem and data at hand, it is natural to experiment with different values of these hyperparameters to try to improve model performance.

> We can think of these **hyperparameters** as _dials_ of the base model

<img width=60% src='images/dials.png'/>

### Difference from Parametric / Non-Parametric Models

Contrast the notion of hyperparameters with the distinction between parametric and non-parametric models.

A linear regression model is parametric in the sense that we start with a given model *form* and we then search for the optimal parameters to fill in that form. But *those* parameters are not the sort we might tweak for the purposes of improving model performance. On the contrary, there is one best set of parameters, and the training of the model is a matter of finding those optimal values.

## Data Example

![Penguins](https://raw.githubusercontent.com/allisonhorst/palmerpenguins/69530276d74b99df81cc385f4e95c644da69ebfa/man/figures/lter_penguins.png)

> Images source: @allison_horst [github.com/allisonhorst/penguins](github.com/allisonhorst/penguins)

In [2]:
penguins = sns.load_dataset('penguins')

![Bill length & depth](https://raw.githubusercontent.com/allisonhorst/palmerpenguins/69530276d74b99df81cc385f4e95c644da69ebfa/man/figures/culmen_depth.png)

> Images source: @allison_horst [github.com/allisonhorst/penguins](github.com/allisonhorst/penguins)

In [3]:
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [4]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [5]:
penguins.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


### Data Prep

We'll try to predict species given the other columns' values. Let's dummy-out `island` and `sex`:

In [6]:
penguins.isna().sum().sum()

19

In [7]:
penguins = penguins.dropna()

In [8]:
y = penguins.pop('species')
y

0      Adelie
1      Adelie
2      Adelie
4      Adelie
5      Adelie
        ...  
338    Gentoo
340    Gentoo
341    Gentoo
342    Gentoo
343    Gentoo
Name: species, Length: 333, dtype: object

In [9]:
penguins

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...
338,Biscoe,47.2,13.7,214.0,4925.0,Female
340,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Biscoe,45.2,14.8,212.0,5200.0,Female


In [10]:
# Note we're dedicating a lot of data to the testing set just for demonstrative purposes
X_train, X_test, y_train, y_test = train_test_split(
    penguins, y, test_size=0.5, random_state=42)

In [41]:
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(sparse_output=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names_out(),
                       index=X_train_cat.index)

In [42]:
dums_df.head()

,island_Biscoe,island_Dream,island_Torgersen,sex_Female,sex_Male
160,0.0,1.0,0.0,1.0,0.0
237,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0
121,0.0,0.0,1.0,0.0,1.0
179,0.0,1.0,0.0,0.0,1.0


In [44]:
X_train_nums = X_train.select_dtypes('float64')

ss = StandardScaler()

ss.fit(X_train_nums)
nums_df = pd.DataFrame(ss.transform(X_train_nums),
                       columns= X_train_nums.columns,
                      index=X_train_nums.index)
nums_df

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
160,0.362748,0.903276,-0.472344,-0.094599
237,0.973499,-0.977375,1.408317,2.512546
2,-0.725152,0.445820,-0.472344,-1.185963
121,-1.221387,1.360731,-0.255345,-0.882806
179,1.030757,0.954104,-0.110678,-0.519018
...,...,...,...,...
194,1.297960,1.004932,-0.400011,-0.822175
77,-1.316816,1.157417,-1.268008,-0.397756
112,-0.839667,0.293335,-0.617010,-1.246594
277,0.267318,-1.079032,1.335984,0.936132


In [45]:
X_train_clean = pd.concat([nums_df, dums_df], axis=1)

In [46]:
X_train_clean.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_Biscoe,island_Dream,island_Torgersen,sex_Female,sex_Male
160,0.362748,0.903276,-0.472344,-0.094599,0.0,1.0,0.0,1.0,0.0
237,0.973499,-0.977375,1.408317,2.512546,1.0,0.0,0.0,0.0,1.0
2,-0.725152,0.445820,-0.472344,-1.185963,0.0,0.0,1.0,1.0,0.0
121,-1.221387,1.360731,-0.255345,-0.882806,0.0,0.0,1.0,0.0,1.0
179,1.030757,0.954104,-0.110678,-0.519018,0.0,1.0,0.0,0.0,1.0


#### Preparing the Test Set

In [47]:
X_test_cat = X_test.select_dtypes('object')

test_dums = ohe.transform(X_test_cat)
test_dums_df = pd.DataFrame(test_dums,
                       columns=ohe.get_feature_names_out(),
                      index=X_test_cat.index)

In [51]:
X_test_nums = X_test.select_dtypes('float64')

test_nums = ss.transform(X_test_nums)
test_nums_df = pd.DataFrame(test_nums,
                            columns=X_test_nums.columns,
                           index=X_test_nums.index)

In [52]:
X_test_clean = pd.concat([test_nums_df,
                 test_dums_df], axis=1)

In [53]:
X_test_clean.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_Biscoe,island_Dream,island_Torgersen,sex_Female,sex_Male
30,-0.877839,-0.214949,-1.702007,-1.185963,0.0,1.0,0.0,1.0,0.0
317,0.534522,-1.282345,1.480650,0.784554,1.0,0.0,0.0,1.0,0.0
79,-0.381604,1.004932,-0.472344,-0.276493,0.0,0.0,1.0,0.0,1.0
201,1.088015,0.090021,-0.255345,-0.670597,0.0,1.0,0.0,1.0,0.0
63,-0.572464,0.547477,-0.689343,-0.215862,1.0,0.0,0.0,0.0,1.0


### Trying Different Models & Values

#### Decision Tree

In [54]:
dt = DecisionTreeClassifier(random_state=10)

dt.fit(X_train_clean, y_train)

DecisionTreeClassifier(random_state=10)

In [55]:
dt.score(X_test_clean, y_test)

0.9700598802395209

##### Changing the branching criterion

In [56]:
dt2 = DecisionTreeClassifier(criterion='entropy',
                          random_state=10)

dt2.fit(X_train_clean, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=10)

In [57]:
dt2.score(X_test_clean, y_test)

0.9700598802395209

##### Changing the max_depth

In [58]:
# How much tho?
dt3 = DecisionTreeClassifier(max_depth=None, criterion='entropy',
                          random_state=10)

dt3.fit(X_train_clean, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=10)

In [59]:
dt3.score(X_test_clean, y_test)

0.9700598802395209

# Automatically Searching with Grid Search

It's not a bad idea to experiment with the values of your models' hyperparameters a bit as you're getting a feel for your models' performance. But there are more systematic ways of going about the search for optimal hyperparameters. One method of hyperparameter tuning is **grid searching**. 

The idea is to build multiple models with different hyperparameter values and then see which one performs the best. The hyperparameters and the values to try form a sort of *grid* along which we are looking for the best performance. For example:


    1           | 'minkowski' | 'uniform'
    3           | 'manhattan' | 'distance'
    5           |
    ______________________________________
    n_neighbors | metric      | weights

Scikit-Learn has a [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class whose `fit()` method runs this procedure. Note that this can be quite computationally expensive since:

- A model is constructed for each combination of hyperparameter values that we input; and
- Each model is cross-validated.

In [60]:
3 * 2 * 2 * 5

60

### `GridSearchCV`

In [61]:
# Define the parameter grid

grid = {'max_depth': [None, 2, 3, 4, 5], 
        'min_samples_split': [2, 3, 4, 5], 
        'min_samples_leaf': [1, 2, 3, 4]}

**Question: How many models will we be constructing with this grid?**

In [62]:
dt_model = DecisionTreeClassifier(random_state=10)

In [63]:
# Initialize the grid search object with five-fold cross-validation

gs = GridSearchCV(estimator=dt_model, param_grid=grid, verbose=3)

In [64]:
gs.fit(X_train_clean, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=1.000 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.939 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=1.000 total time=   0.0s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.939 total time=   0.0s
[CV 5/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=1.000 total time=   0.0s
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=3;, score=1.000 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=3;, score=0.939 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=3;, score=1.000 total time=   0.0s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=3;, score=0.939 total time=   0.0s
[CV 5/5] END max_dep

GridSearchCV(estimator=DecisionTreeClassifier(random_state=10),
             param_grid={'max_depth': [None, 2, 3, 4, 5],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 3, 4, 5]},
             verbose=3)

In [65]:
gs.best_params_

{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [66]:
gs.best_score_

0.9757575757575758

In [67]:
gs.best_estimator_

DecisionTreeClassifier(random_state=10)

In [68]:
gs.cv_results_

{'mean_fit_time': array([0.00435443, 0.00205464, 0.00167327, 0.0014214 , 0.00105848,
        0.00129924, 0.00127821, 0.0010428 , 0.0012897 , 0.00104604,
        0.00108538, 0.00117483, 0.00111222, 0.0010704 , 0.00106626,
        0.00119781, 0.0011045 , 0.00132899, 0.00144963, 0.00108609,
        0.00128307, 0.00123544, 0.00104275, 0.00115113, 0.0013751 ,
        0.0010632 , 0.00105929, 0.00125232, 0.00113997, 0.00106335,
        0.00132709, 0.00119805, 0.00114489, 0.00113144, 0.00128121,
        0.00112004, 0.00108132, 0.00147672, 0.00138578, 0.001263  ,
        0.00134935, 0.00118542, 0.00109639, 0.00124712, 0.00112305,
        0.001057  , 0.00126858, 0.0011054 , 0.00109148, 0.0012876 ,
        0.00111675, 0.0011097 , 0.00117435, 0.00119843, 0.00115886,
        0.00104151, 0.00153303, 0.00121288, 0.0011024 , 0.00126767,
        0.00108938, 0.00104141, 0.00151176, 0.00102921, 0.00121336,
        0.00135818, 0.00108657, 0.00111184, 0.00135236, 0.00110226,
        0.0012248 , 0.00104723,

In [69]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004354,0.003276,0.001898,0.000725,None,1,2,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",1.000000,0.939394,1.000000,0.939394,1.0,0.975758,0.029691,1
1,0.002055,0.000204,0.001165,0.000110,None,1,3,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",1.000000,0.939394,1.000000,0.939394,1.0,0.975758,0.029691,1
2,0.001673,0.000170,0.000975,0.000120,None,1,4,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",1.000000,0.939394,1.000000,0.939394,1.0,0.975758,0.029691,1
3,0.001421,0.000268,0.000736,0.000059,None,1,5,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",1.000000,0.939394,1.000000,0.939394,1.0,0.975758,0.029691,1
4,0.001058,0.000033,0.000654,0.000048,None,2,2,"{'max_depth': None, 'min_samples_leaf': 2, 'mi...",0.970588,0.939394,0.969697,0.939394,1.0,0.963815,0.022727,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.001338,0.000353,0.000697,0.000073,5,3,5,"{'max_depth': 5, 'min_samples_leaf': 3, 'min_s...",1.000000,0.939394,0.969697,0.909091,1.0,0.963636,0.035339,25
76,0.001212,0.000146,0.000687,0.000047,5,4,2,"{'max_depth': 5, 'min_samples_leaf': 4, 'min_s...",1.000000,0.878788,0.969697,0.909091,1.0,0.951515,0.049237,49
77,0.001104,0.000039,0.000658,0.000027,5,4,3,"{'max_depth': 5, 'min_samples_leaf': 4, 'min_s...",1.000000,0.878788,0.969697,0.909091,1.0,0.951515,0.049237,49
78,0.001309,0.000322,0.000786,0.000272,5,4,4,"{'max_depth': 5, 'min_samples_leaf': 4, 'min_s...",1.000000,0.878788,0.969697,0.909091,1.0,0.951515,0.049237,49


### Choice of Grid Values

Which values should you pick for your grid? Intuitively, you should try both "large" and "small" values, but of course what counts as large and small will really depend on the type of hyperparameter.

- ALWAYS INCLUDE THE DEFAULT IN YOUR FIRST SEARCH
- For a k-nearest neighbors model, 1 or 3 would be a small value for the number of neighbors and 15 or 17 would be a large value.
- For a decision tree model, what counts as a small `max_depth` will really depend on the size of your training data. A `max_depth` of 5 would likely have little effect on a very small dataset but, at the same time, it would probably significantly decrease the variance of a model where the dataset is large.
- For a logistic regression's regularization constant, you may want to try a set of values that are exponentially separated, like \[1, 10, 100, 1000\].
- **If a grid search finds optimal values at the ends of your hyperparameter ranges, you might try another grid search with more extreme values.**

In [70]:
grid_1 = {'max_depth': [1, 3, 5], 'min_samples_split': [2, 3, 4, 5, 6]}

# Best values were 5 and 6

grid_2 = {'max_depth': [5, 10, 15], 'min_samples_split': [6, 8, 10]}

# New best values were 10 and 8

grid_3 = {'max_depth': [8, 9, 10, 11, 12], 'min_samples_split': [7, 8, 9]}

# Better Process: Pipelines

> **Pipelines** can keep our code neat and clean all the way from gathering & cleaning our data, to creating models & fine-tuning them!

![](https://imgs.xkcd.com/comics/data_pipeline.png)

The `Pipeline` class from [Scikit-Learn's API](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) is especially convenient since it allows us to use our other Estimators that we know and love!

## Advantages of `Pipeline`

### Reduces Complexity

> You can focus on particular parts of the pipeline one at a time and debug or adjust parts as needed.

### Convenient

> The pipeline summarizes your fine-detail steps. That way you can focus on the big-picture aspects.

### Flexible

> You can use pipelines with different models and with GridSearch.

### Prevent Mistakes

> We can focus on one section at a time.
>
> We also can ensure data leakage between our training and doesn't occur between our training dataset and validation/testing datasets!

## Example of Using `Pipeline`

In [71]:
# Getting some data
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=27)

### Without the Pipeline class

In [72]:
# Define transformers (will adjust/massage the data)
imputer = SimpleImputer(strategy="median") # replaces missing values
std_scaler = StandardScaler() # scales the data

# Define the classifier (predictor) to train
rf_clf = DecisionTreeClassifier(random_state=42)

# Have the classifer (and full pipeline) learn/train/fit from the data
X_train_filled = imputer.fit_transform(X_train)
X_train_scaled = std_scaler.fit_transform(X_train_filled)
rf_clf.fit(X_train_scaled, y_train)

# Predict using the trained classifier (still need to do the transformations)
X_test_filled = imputer.transform(X_test)
X_test_scaled = std_scaler.transform(X_test_filled)
y_pred = rf_clf.predict(X_test_scaled)
print(y_pred)

[2 0 2 2 1 1 1 2 2 0 1 1 0 1 1 1 0 2 2 2 1 0 1 0 2 2 1 0 1 2]


> Note that if we were to add more steps in this process, we'd have to change both the *training* and *testing* processes.

### With `Pipeline` Class

In [73]:
pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), 
        ('std_scaler', StandardScaler()),
        ('rf_clf', DecisionTreeClassifier(random_state=42)),
])


# Train the pipeline (tranformations & predictor)
pipeline.fit(X_train, y_train)

# Predict using the pipeline (includes the transfomers & trained predictor)
predicted = pipeline.predict(X_test)
print(predicted)

[2 0 2 2 1 1 1 2 2 0 1 1 0 1 1 1 0 2 2 2 1 0 1 0 2 2 1 0 1 2]


In [74]:
pipeline['imputer']

SimpleImputer(strategy='median')

In [75]:
pipeline.named_steps

{'imputer': SimpleImputer(strategy='median'),
 'std_scaler': StandardScaler(),
 'rf_clf': DecisionTreeClassifier(random_state=42)}

In [92]:
pipeline['rf_clf'].feature_importances_

array([0.        , 0.03338202, 0.55667565, 0.40994234])

> If we need to change our process, we change it _just once_ in the Pipeline

## Grid Searching a Pipeline

> Let's first get our data prepared like we did before

In [ ]:
penguins = sns.load_dataset('penguins')
penguins = penguins.dropna()

In [ ]:
y = penguins.pop('species')
X_train, X_test, y_train, y_test = train_test_split(
    penguins, y, test_size=0.5, random_state=42)

In [ ]:
X_train_nums = X_train.select_dtypes('float64')

ss = StandardScaler()

ss.fit(X_train_nums)
nums_df = pd.DataFrame(ss.transform(X_train_nums),
                      index=X_train_nums.index)

In [ ]:
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(sparse=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_train_cat.index)

> Intermediary step to treat categorical and numerical data differently

### Using `ColumnTransformer`

In [ ]:
X_train_nums.columns

In [ ]:
numerical_pipeline = Pipeline(steps=[('ss', StandardScaler())])

categorical_pipeline = Pipeline(steps=[('ohe', OneHotEncoder(sparse=False, 
                                                            handle_unknown='ignore'))])
transformer = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, X_train_nums.columns),
    ('cat', categorical_pipeline, X_train_cat.columns)])

In [ ]:
model_pipe = Pipeline(steps=[('col_tr', transformer),
                             ('log_reg', LogisticRegression(random_state=42))])

> Finally showing we can fit the full pipeline

In [ ]:
model_pipe.fit(X_train, y_train)

In [ ]:
model_pipe.score(X_train, y_train)

In [ ]:
model_pipe.score(X_test, y_test)

> Performing grid search on the full pipeline

In [ ]:
model_pipe.named_steps

In [ ]:
model_pipe.named_steps['log_reg']

In [ ]:
model_pipe['col_tr'].named_transformers_

In [ ]:
pipe_grid = None
gs_pipe = GridSearchCV(estimator=model_pipe, param_grid=pipe_grid, verbose=2)

In [ ]:
gs_pipe.fit(X_train, y_train)

In [ ]:
pd.DataFrame(gs_pipe.cv_results_)

In [ ]:
gs_pipe.best_params_

In [ ]:
gs_pipe.best_estimator_.score(X_test, y_test)

# Grid Search Exercise

Use a classifier of your choice to predict the category of price range for the phones in this dataset. Try tuning some hyperparameters using a grid search, and then write up a short paragraph about your findings.

In [ ]:
phones_train = pd.read_csv('data/train.csv')
phones_test = pd.read_csv('data/test.csv')

# Level Up: Random Searching

It is also possible to search for good hyperparameter values randomly. This is a nice choice if computation time is an issue or if you are tuning over continuous hyperparameters.

### `RandomizedSearchCV` with `LogisticRegression`

In [ ]:
log_reg_grid = {'C': stats.uniform(loc=0, scale=10),
               'l1_ratio': stats.expon(scale=0.2)}

In [ ]:
rs = RandomizedSearchCV(estimator=LogisticRegression(penalty='elasticnet',
                                                    solver='saga',
                                                    max_iter=1000,
                                                    random_state=42),
                        param_distributions=log_reg_grid,
                       random_state=42, n_iter=100, verbose=2)

rs.fit(X_train_clean, y_train)

rs.best_params_